In [2]:
import pandas as pd
import openpyxl
import os

In [3]:
health_df = pd.read_excel('../data/calenviroscreen40resultsdatadictionary_F_2021.xlsx', sheet_name=0)

In [4]:
def make_decimal(tract_str):
    num = tract_str[4:-2]
    decimal = tract_str[-2:]
    if decimal != '00':
        return num + '.' + decimal
    else:
        return num
        
health_df['tract'] = health_df['Census Tract'].astype(str).apply(make_decimal)

In [5]:
ssf_tracts = [
    '6001P',
    '6017',
    '6018P',
    '6018',
    '6019',
    '6019.01',
    '6019.02',
    '6020P',
    '6020.01',
    '6020.02',
    '6021',
    '6022',
    '6022.02',
    '6022.01',
    '6023P',
    '6024P',
    '6024',
    '6025',
    '6026',
    '6026.02',
    '6038.01',
    '6038.02',
    '6039',
    '6040',
    '6041.02P',
    '6041.03',
    '6041.04',
    '6042P'
]
sb_tracts = [
    '6023P',
    '6030P',
    '6035P',
    '6036P',
    '6037',
    '6038P',
    '6039P',
    '6040',
    '6041.01P',
    '6041.02P',
    '6042P',
    '6046P',
    '6135.01P'
]

other_tracts = [
    '6016.01',
    '6027',
    '6140',
    '6016.03',
    '6016.05'
]

tracts = ssf_tracts + sb_tracts + other_tracts

# Tracts in the shapefile don't contain 'P'
reformat = lambda tract: tract.replace('P', '')
tracts_reformatted = list(map(reformat, tracts))

In [6]:
health_df = health_df[health_df['tract'].isin(tracts_reformatted)][health_df['California County'] == 'San Mateo']
health_df = health_df.drop(columns=['Census Tract', 'California County', 'ZIP', 'Approximate Location', 'Longitude', 'Latitude'])

C:\Users\isaac\AppData\Local\Temp\ipykernel_35024\2904978862.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  health_df = health_df[health_df['tract'].isin(tracts_reformatted)][health_df['California County'] == 'San Mateo']


In [11]:
indicators = pd.DataFrame({
    'tract': tracts_reformatted
})

def find_max(periods):
    def extract_year(p):
        if type(p) == str:
            return int(p.split('-')[-1])
        else:
            return p
    
    
    max_y = -1
    max_i = -1
    
    for i, p in enumerate(periods):
        y = extract_year(p)
        
        if max_y < 0 or max_y < y:
            max_y = y
            max_i = i
            
    return periods[max_i]

def most_recent(group):
    latest_measurement = find_max(group['Period of Measure'].unique())
    latest = group[group['Period of Measure'] == latest_measurement]
    return latest.iloc[0]

for indicator_file in os.listdir('../data/smc_indicators/'):
    indicator = indicator_file.split('.')[0]

    indicator_data = pd.read_csv(f'../data/smc_indicators/{indicator_file}')
    indicator_data['tract'] = indicator_data['Location'].astype(str).apply(make_decimal)
    
    indicator_cleaned = indicator_data[indicator_data['tract'].isin(tracts_reformatted)].copy()
    indicator_cleaned = indicator_cleaned.groupby(['tract']).apply(most_recent).reset_index(drop=True)
    
    indicator_cleaned[indicator] = indicator_cleaned['Indicator Rate Value']
    indicator_only = indicator_cleaned[['tract', indicator]]
    
    indicators = pd.merge(indicators, indicator_only, on='tract', how='left')

Adults who are Sedentary.csv
Adults who have had a Routine Checkup.csv
Adults with COPD.csv
Children with Low Access to a Grocery Store (% of Total Pop).csv
Health Equity Index.csv
Income Inequality.csv
Low-Income and Low Access to a Grocery Store.csv
Overcrowded Households.csv
Self-Reported General Health Assessment Poor or Fair.csv
Workers Commuting by Public Transportation.csv
Workers who Walk to Work.csv


In [12]:
all_indicator_data = pd.merge(indicators, health_df, on='tract', how='left')

In [13]:
all_indicator_data

,tract,Adults who are Sedentary,Adults who have had a Routine Checkup,Adults with COPD,Children with Low Access to a Grocery Store (% of Total Pop),Health Equity Index,Income Inequality,Low-Income and Low Access to a Grocery Store,Overcrowded Households,Self-Reported General Health Assessment Poor or Fair,...,Linguistic Isolation Pctl,Poverty,Poverty Pctl,Unemployment,Unemployment Pctl,Housing Burden,Housing Burden Pctl,Pop. Char.,Pop. Char. Score,Pop. Char. Pctl
0,6001,17.5,70.2,4.6,16.2,22.6,0.560,12.4,4.0,13.7,...,35.336311,12.2,15.025126,2.4,9.721499,17.5,51.774398,39.213225,4.067686,34.140192
1,6017,20.2,70.6,4.1,0.0,1.0,0.327,0.0,1.7,15.3,...,30.694275,12.8,16.557789,2.9,15.838105,14.0,33.206591,46.317132,4.804592,44.704992
2,6018,18.5,71.3,4.1,0.0,5.8,0.437,0.0,1.6,13.5,...,53.363107,9.6,9.271357,1.0,1.340448,16.3,45.551331,37.801404,3.921234,31.782653
3,6018,18.5,71.3,4.1,0.0,5.8,0.437,0.0,1.6,13.5,...,53.363107,9.6,9.271357,1.0,1.340448,16.3,45.551331,37.801404,3.921234,31.782653
4,6019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6019.01,22.5,69.6,4.8,0.0,9.8,0.477,0.0,4.3,17.5,...,64.812559,17.5,28.417085,4.1,32.274857,10.1,14.245881,54.615895,5.665443,57.047403
6,6019.02,18.2,68.2,3.5,0.0,2.6,0.372,0.0,11.3,13.6,...,55.948031,14.6,20.954774,0.5,0.325351,13.4,30.202788,40.946281,4.247460,36.661624
7,6020,19.8,63.2,4.2,1.1,NaN,0.414,0.8,NaN,14.6,...,50.006767,20.1,34.484925,3.2,19.586153,20.0,62.826362,50.936117,5.283731,51.802824
8,6020.01,19.4,70.1,4.4,NaN,3.5,0.382,NaN,5.5,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,6020.02,19.0,70.0,3.5,NaN,4.6,0.381,NaN,1.2,14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
all_indicator_data.to_csv('../data/all_indicators.csv', index=False)